<a href="https://colab.research.google.com/github/Krishnaugale353/Chatbot/blob/main/ChatBot_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install langchain
!pip install openai
!pip install langchain_community
!pip install chromadb
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.0 MB/s eta 0:00:00

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.3 MB/s eta 0:00:00


In [1]:
from google.colab import userdata
import os
openai_key = userdata.get('OPENAI')
os.environ['OPENAI_API_KEY'] = openai_key

In [ ]:
# User 1:
#         session 1: data science
#         session 2: russia ukraine
# user 2:
#         session 1
#         session 2

In [2]:
import os
from typing import Dict, List, Any, Optional
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.llms import OpenAI

class ChatbotManager:
    """A manager class for handling multiple chatbot sessions across different users."""

    def __init__(self, window_size: int = 4, temperature: float = 0.7):
        """
        Initialize the ChatbotManager.

        Args:
            api_key: OpenAI API key (if None, will try to use environment variable)
            window_size: Number of conversation exchanges to keep in memory
            temperature: Temperature parameter for the LLM (higher = more creative)
        """
        # Set API key if provided, otherwise use environment variable


        self.window_size = window_size
        self.temperature = temperature
        self.user_data: Dict[str, Dict[str, Any]] = {}

    def create_user(self, user_id: str, user_name: str) -> None:
        """
        Create a new user with empty session histories.

        Args:
            user_id: Unique identifier for the user
            user_name: Name of the user
        """
        self.user_data[user_id] = {
            "name": user_name,
            "history": {}
        }

    def create_session(self, user_id: str, session_id: str) -> None:
        """
        Create a new session for an existing user.

        Args:
            user_id: User identifier
            session_id: Unique session identifier

        Raises:
            ValueError: If the user doesn't exist
        """
        if user_id not in self.user_data:
            raise ValueError(f"User '{user_id}' does not exist")

        self.user_data[user_id]["history"][session_id] = []

    def get_user_history(self, user_id: str, session_id: str) -> List:
        """
        Retrieve conversation history for a specific user session.

        Args:
            user_id: User identifier
            session_id: Session identifier

        Returns:
            List of conversation messages

        Raises:
            ValueError: If the user or session doesn't exist
        """
        if user_id not in self.user_data:
            raise ValueError(f"User '{user_id}' does not exist")

        if session_id not in self.user_data[user_id]["history"]:
            raise ValueError(f"Session '{session_id}' does not exist for user '{user_id}'")

        return self.user_data[user_id]["history"][session_id]

    def store_history(self, user_id: str, session_id: str, history: List) -> None:
        """
        Store conversation history for a specific user session.

        Args:
            user_id: User identifier
            session_id: Session identifier
            history: List of conversation messages

        Raises:
            ValueError: If the user or session doesn't exist
        """
        if user_id not in self.user_data:
            raise ValueError(f"User '{user_id}' does not exist")

        if session_id not in self.user_data[user_id]["history"]:
            raise ValueError(f"Session '{session_id}' does not exist for user '{user_id}'")

        self.user_data[user_id]["history"][session_id] = history

    def chat(self, user_id: str, session_id: str, message: str) -> Dict[str, Any]:
        """
        Process a user message and return the chatbot's response.

        Args:
            user_id: User identifier
            session_id: Session identifier
            message: User's input message

        Returns:
            Dictionary containing the response and related information

        Raises:
            ValueError: If the user or session doesn't exist
        """
        # Create memory and chain for this interaction
        memory = ConversationBufferWindowMemory(k=self.window_size, return_messages=True)

        # Initialize the LLM and conversation chain
        llm = OpenAI(temperature=self.temperature)
        chain = ConversationChain(
            llm=llm,
            memory=memory,
            verbose=False  # Set to True for debugging
        )

        # Load previous history if it exists
        try:
            history = self.get_user_history(user_id, session_id)
            memory.chat_memory.messages = history
        except ValueError as e:
            # Create user and session if they don't exist
            if user_id not in self.user_data:
                self.create_user(user_id, user_id)  # Use ID as default name
            if session_id not in self.user_data[user_id]["history"]:
                self.create_session(user_id, session_id)

        # Process the message
        response = chain.invoke(message)

        # Update and store the history
        history = memory.chat_memory.messages
        self.store_history(user_id, session_id, history)

        return response


# Example usage:
if __name__ == "__main__":
    # Initialize the chatbot manager
    chatbot = ChatbotManager()

    # Create users
    chatbot.create_user("user_1", "Alice")
    chatbot.create_user("user_2", "Bob")

    # Create sessions
    chatbot.create_session("user_1", "session_1")
    chatbot.create_session("user_1", "session_2")
    chatbot.create_session("user_2", "session_1")

    # Example conversation with user_1, session_1
    response1 = chatbot.chat("user_1", "session_1", "What is data science?")
    print(f"Response: {response1['response']}")

    response2 = chatbot.chat("user_1", "session_1", "Is it a good career choice?")
    print(f"Response: {response2['response']}")

    # Example conversation with user_1, session_2
    response3 = chatbot.chat("user_1", "session_2", "Tell me about Russia and Ukraine")
    print(f"Response: {response3['response']}")

    # Continue conversation in session_1
    response4 = chatbot.chat("user_1", "session_1", "What skills do I need for data science?")
    print(f"Response: {response4['response']}")

<ipython-input-2-802388152b8a>:113: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=self.window_size, return_messages=True)
<ipython-input-2-802388152b8a>:116: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=self.temperature)
<ipython-input-2-802388152b8a>:117: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.


Response:  Data science is an interdisciplinary field that combines computational and statistical methods to extract knowledge and insights from data. It involves collecting, organizing, analyzing, and interpreting large amounts of data to make informed decisions and predictions. Data science has applications in various industries, including business, healthcare, and technology. It is also used for problem-solving and decision-making in fields such as finance, marketing, and social sciences.
Response:  Yes, data science is a rapidly growing field with a high demand for skilled professionals. It offers a variety of job opportunities and competitive salaries. However, it also requires continuous learning and keeping up with new technologies and techniques. So, it can be a good career choice for those who are interested in data and analytics and are willing to invest in their education and skills.
Response:  Russia and Ukraine are two neighboring countries in Eastern Europe. Russia is the